# Matchup Evaluator
This notebook is used to create code and logic that will help analyse an upcoming h2h Yahoo
Fantasy matchup

In [1]:
from src.data import all_functions as af
import pandas as pd

In [7]:
# Authenticate
sc = af.yahoo_fantasy_api_authentication()
league = af.yahoo_fantasy_league(sc)

[2021-02-04 20:02:17,414 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-04 20:02:17,416 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 609.3742136955261
[2021-02-04 20:02:17,417 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [8]:
# Nunn 9Cat averages
nunn_of_yall_betta_team = af.team_9cat_average_stats(af.NUNN_OF_YALL_BETTA, league)

In [9]:
nunn_of_yall_betta_team = nunn_of_yall_betta_team.loc["mean"]
nunn_of_yall_betta_team["PLAYER"] = af.NUNN_OF_YALL_BETTA["name"]
nunn_of_yall_betta_team = pd.DataFrame(nunn_of_yall_betta_team).T
nunn_of_yall_betta_team

,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
mean,Nunn of Y'all Betta,0.462,0.817,1.95,15.17,4.99,3.68,0.99,0.37,1.67


In [11]:
# Crabbe 9Cat averages
crabbeherbythepussy_team = af.team_9cat_average_stats(af.CRABBEHERBYTHEPUSSY, league)

In [12]:
crabbeherbythepussy_team = crabbeherbythepussy_team.loc["mean"]
crabbeherbythepussy_team["PLAYER"] = af.CRABBEHERBYTHEPUSSY["name"]
crabbeherbythepussy_team = pd.DataFrame(crabbeherbythepussy_team).T
crabbeherbythepussy_team

,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
mean,CrabbeHerByThePussy,0.486,0.817,1.52,14.22,6.17,3.02,0.88,0.92,1.63


In [18]:
# Drop PLAYER
nunn_of_yall_betta_team.drop("PLAYER", inplace=True, axis=1)
crabbeherbythepussy_team.drop("PLAYER", inplace=True, axis=1)

In [21]:
# Function to convert dataframe columns to dtype floats
def columns_to_dtype_float(dataframe):
    """
    Converts dataframe columns to dtype float.
    param dataframe (pandas dataframe): dataframe to change column types
    returns dataframe (pandas dataframe): dataframe with changed column types
    """
    for column in dataframe.columns:
        dataframe[column] = dataframe[column].astype(float)
    return dataframe

In [22]:
nunn_of_yall_betta_team = columns_to_dtype_float(nunn_of_yall_betta_team)
crabbeherbythepussy_team = columns_to_dtype_float(crabbeherbythepussy_team)

In [25]:
matchup_difference = nunn_of_yall_betta_team.sub(crabbeherbythepussy_team)
matchup_difference

,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
mean,-0.024,0.0,0.43,0.95,-1.18,0.66,0.11,-0.55,0.04


In [20]:
def naive_win_projection(dataframe):
    """

    """
    sum_of_9cat_differences = matchup_difference.sum(axis=1)

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, mean to mean
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   FG_PCT  1 non-null      object
 1   FT_PCT  1 non-null      object
 2   FG3M    1 non-null      object
 3   PTS     1 non-null      object
 4   REB     1 non-null      object
 5   AST     1 non-null      object
 6   STL     1 non-null      object
 7   BLK     1 non-null      object
 8   TOV     1 non-null      object
dtypes: object(9)
memory usage: 80.0+ bytes


In [30]:
matchup_difference.sum(axis=1)

mean    0.436
dtype: float64

In [ ]:
COLUMN_9CAT_DECIMAL_FORMAT = {af.FIELD_GOAL_PERCENTAGE_COLUMN : af.p9ca.THREE_DECIMAL_FORMAT,
     af.FREE_THROW_PERCENTAGE_COLUMN : af.p9ca.THREE_DECIMAL_FORMAT,
     af.THREES_MADE_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT,
     af.POINTS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT,
     af.REBOUNDS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT,
     af.ASSITS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT, af.STEALS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT,
     af.BLOCKS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT, af.TURNOVERS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT}

def applymap_color_and_format(dataframe):
    """

    """
    fromated_matchup_difference = matchup_difference.style.applymap(
        af.color_negative_red, subset=pd.IndexSlice[:,[af.FIELD_GOAL_PERCENTAGE_COLUMN,
                                                       af.FREE_THROW_PERCENTAGE_COLUMN,
                                                       af.THREES_MADE_COLUMN, af.POINTS_COLUMN,
                                                       af.REBOUNDS_COLUMN, af.ASSITS_COLUMN,
                                                       af.STEALS_COLUMN, af.BLOCKS_COLUMN
                                                       ]]).applymap(
        af.color_negative_red_tov, subset=pd.IndexSlice[:,[af.TURNOVERS_COLUMN]]).format(COLUMN_9CAT_DECIMAL_FORMAT)
    return fromated_matchup_difference

In [38]:
matchup_difference.style.applymap(
    af.color_negative_red, subset=pd.IndexSlice[:,["FG_PCT", "FT_PCT", "FG3M", "PTS", "REB", "AST",
                                                "STL", "BLK"]]).applymap(
    af.color_negative_red_tov, subset=pd.IndexSlice[:,["TOV"]]).format(a)

,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
mean,-0.024,0.000,0.43,0.95,-1.18,0.66,0.11,-0.55,0.04


In [66]:
PLAYER_COLUMN = "PLAYER"
MEAN_ROW = "mean"
NAME_KEY = "name"
COLUMN_9CAT_DECIMAL_FORMAT = {af.FIELD_GOAL_PERCENTAGE_COLUMN : af.p9ca.THREE_DECIMAL_FORMAT,
     af.FREE_THROW_PERCENTAGE_COLUMN : af.p9ca.THREE_DECIMAL_FORMAT,
     af.THREES_MADE_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT,
     af.POINTS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT,
     af.REBOUNDS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT,
     af.ASSITS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT, af.STEALS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT,
     af.BLOCKS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT, af.TURNOVERS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT}

def columns_to_dtype_float(dataframe):
    """
    Converts dataframe columns to dtype float.
    param dataframe (pandas dataframe): dataframe to change column types
    returns dataframe (pandas dataframe): dataframe with changed column types
    """
    for column in dataframe.columns:
        dataframe[column] = dataframe[column].astype(float)
    return dataframe

def get_and_format_team_9cat(team, league):
    """
    Function first finds the teams 9cat averages by calling the *team_9cat_average_stats*
    function. It then filters to find the team mean and cleans and formats this single row
    dataframe. It changes the dtype of the columns to be floats.
    param team (dictionary): team dictionary
    param league (yahoo oauth object): yahoo authentication object
    returns team_9cat (dataframe): dataframe of team average 9cats
    """
    team_9cat = af.team_9cat_average_stats(team, league)
    team_9cat = team_9cat.loc[MEAN_ROW]
    team_9cat[PLAYER_COLUMN] = team[NAME_KEY]
    team_9cat = pd.DataFrame(team_9cat).T
    team_9cat.drop(PLAYER_COLUMN, inplace=True, axis=1)
    team_9cat = columns_to_dtype_float(team_9cat)
    return team_9cat

def applymap_color_and_format(dataframe):
    """
    Function takes a dataframe containing 9cat stat columns and returns a color coded and decimal
    place formatted dataframe for visual ease.
    param dataframe (pandas dataframe): dataframe to format
    returns dataframe (pandas dataframe): dataframe with formatted and colored values
    """
    colored_matchup_difference = matchup_difference.style.applymap(
        af.color_negative_red, subset=pd.IndexSlice[:,[af.FIELD_GOAL_PERCENTAGE_COLUMN,
                                                       af.FREE_THROW_PERCENTAGE_COLUMN,
                                                       af.THREES_MADE_COLUMN, af.POINTS_COLUMN,
                                                       af.REBOUNDS_COLUMN, af.ASSITS_COLUMN,
                                                       af.STEALS_COLUMN, af.BLOCKS_COLUMN
                                                       ]]).applymap(
        af.color_negative_red_tov, subset=pd.IndexSlice[:,[af.TURNOVERS_COLUMN]]).format\
        (COLUMN_9CAT_DECIMAL_FORMAT)
    return colored_matchup_difference

def compare_h2h_team_9cat(team1, team2):
    """

    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team1_9cat_stats = get_and_format_team_9cat(team1, league)
    team2_9cat_stats = get_and_format_team_9cat(team2, league)
    matchup_difference = team1_9cat_stats.sub(team2_9cat_stats)
    colored_matchup_difference = applymap_color_and_format(matchup_difference)
    return matchup_difference, colored_matchup_difference

In [67]:
matchup_difference, colored_matchup_difference = compare_h2h_team_9cat(af.NUNN_OF_YALL_BETTA, af
                                                                       .CRABBEHERBYTHEPUSSY)

[2021-02-04 22:57:09,050 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-04 22:57:09,055 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2266.9832198619843
[2021-02-04 22:57:09,056 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [68]:
colored_matchup_difference



,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
mean,-0.024,0.000,0.43,0.95,-1.18,0.66,0.11,-0.55,0.04
